# Retail Orders Data Pipeline, Analysis and Visualization using Python, SQL and Tableau

## 1.Introduction

The project extracts, analyzes and visualize data from a dataset of orders of various products made at a retail stationery store by clients from different states. 


The aim of the project is to build an ETL pipeline to extract & transform data in Python, load it to MS SQL Server for analysis and create a dashboard in Tableau.

The SQL Server analysis will involve running queries to answer the following questions:


i) Find the top 5 highest generating products.


ii) Find top 5 highest selling products in each region.


iii) Find month over month growth comparison for 2022 and 2023 sales.


iv) For each category, which month had the highest salaries?

The dashboard will provide an interactive platform to access the summaries of the data. Its key features will include a map to show sales per state and per region, a graph to indicate the month over month for 2022 and 2023 sales and a drop down menu to view comprehensive details of each product.

## 2.Data Extraction and Libraries Import
    
I will start by extracting the data from a csv file and loading packages I will use during this project.

In [13]:
import os
import pandas as pd
#os.getcwd()

'C:\\Users\\user\\Documents\\persona\\PyCharm'

In [15]:
#os.chdir('C:\\Users\\user\\Documents\\persona\\PyCharm')
retail_Data = pd.read_csv('orders.csv')

In [13]:
#os.getcwd()
retail_Data.head(6)         #getting the first 6 rows of the dataset

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5
5,6,2022-03-13,Not Available,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,50,50,7,3


The 'Ship Mode' column has values not available, which is not type of a ship mode.
Im going to check all the levels of ship mode:

In [17]:
retail_Data['Ship Mode'].unique()


array(['Second Class', 'Standard Class', 'Not Available', 'unknown',
       'First Class', nan, 'Same Day'], dtype=object)

Will convert 'Not Available', 'unknown' to nan values such that they can be counted as not a number, which is a recognized type of data.

In [20]:
#reload the csv file with corrected ship modes:
retail_Data = pd.read_csv('orders.csv',na_values = ['Not Available', 'unknown'])


In [22]:
retail_Data['Ship Mode'].unique() 

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)



## 3.Data Transformation and Pre-processing


I will check the structure of our dataset and clean it to prepare it for the loading to the SQL server.


I will change names of variables or data types and derive new columns or remove columns such that the data can comply with the expected standards of the SQL server management system

In [25]:
retail_Data.shape      #the number of rows and columns of our dataframe.

(9994, 16)

The dataset has 9994 observations by 16 columns.

In [28]:
retail_Data.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


I will change the names of the columns, to lower case and replace ' ' with _.

In [30]:
retail_Data.columns = retail_Data.columns.str.lower()
retail_Data.columns = retail_Data.columns.str.replace(' ','_')
retail_Data.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

To be able to answer the SQL questions, I need to add new columns. The data for the new columns will be derived from the existing columns in our dataframe.
I Will add 'discount', 'sale_price' and 'profit'.

In [33]:
retail_Data['discount'] = retail_Data['list_price'] * retail_Data['discount_percent'] * .01


In [35]:
retail_Data['sale_price'] = retail_Data['list_price'] - retail_Data['discount']


In [37]:
retail_Data['profit'] = retail_Data['sale_price'] - retail_Data['cost_price']


In [39]:
retail_Data.head()   # the dataframe now has 3 added columns.

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


Next im going to convert the order_date from object data type to datetime data type

In [43]:
retail_Data.dtypes  #checking the data types of all the variables in our data frame. 'order_date' is an object not a date.

order_id              int64
order_date           object
ship_mode            object
segment              object
country              object
city                 object
state                object
postal_code           int64
region               object
category             object
sub_category         object
product_id           object
cost_price            int64
list_price            int64
quantity              int64
discount_percent      int64
discount            float64
sale_price          float64
profit              float64
dtype: object

In [47]:
retail_Data['order_date'] = pd.to_datetime(retail_Data['order_date']) #converting 'order_date' to a datetime data type.

In [49]:
retail_Data.dtypes     #order_date is now has a datetime datatype

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

I will delete the 'list_price' and 'cost_price' columns in order to normalize our data for the SQL server.

In [53]:
retail_Data.drop(columns=['list_price','cost_price'],inplace=True)


## 4.Data Loading to Database (MS SQL Server)


The sqlalchemy module will be used to connect the python environment to the SQL server.


The retail order data will be transferred to the MS SQL Server Manager to run queries and analyze the data

In [27]:

import sqlalchemy as sal    #will use the sqlalchemy for the connection

In [88]:
engine = sal.create_engine('mssql://DESKTOP-P2K3MK7\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER') #setting the connection parameters.
conn=engine.connect()

#DESKTOP-P2K3MK7\SQLEXPRESS

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\user\AppData\Local\Temp\ipykernel_2476\1504546519.py:1: SyntaxWarning: invalid escape sequence '\S'
  engine = sal.create_engine('mssql://DESKTOP-P2K3MK7\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')


In [90]:
retail_Data.to_sql('retail_orders', con=conn , index=False, if_exists = 'append')

38

## 5.Conclusion

  
In conclusion, the data analysis conducted on the retail orders has provided valuable insights into product sales trends. By effectively connecting the dataset to a SQL database, we enhanced our ability to query and manipulate the data, allowing for more comprehensive analyses. Building an interactive dashboard in Tableau with a comprehensive interface enabled efficient management and analysis of customer purchases.

    
The findings highlight key patterns in purchasing behavior, such as highest selling products, sales per region which can inform inventory management and marketing strategies. Overall, this project underscores the importance of data-driven decision-making in the retail sector.

    
Future work could involve predictive analytics to optimize business strategies and improve customer engagement.
